In [1]:
import numpy as np
import pandas as pd

file_enron = './enron-v1.csv'
enron_data = pd.read_csv(file_enron)
enron_data

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns  # also improves the look of plots
sns.set()  # set Seaborn defaults
plt.rcParams['figure.figsize'] = [10, 5]  # default hor./vert. size of plots, in inches
plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after import seaborn

import plotly.graph_objects as go
import networkx as nx
from pyvis.network import Network
from pyvis import network as net

# Here is my additional stuff
import plotly.express as px
from bokeh.layouts import column, layout, row, Spacer
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter, 
                          Label, ImageURL, RadioGroup, Button, Select,
                          Arrow, NormalHead, LabelSet)
from bokeh.plotting import figure, curdoc
from bokeh.transform import transform, linear_cmap

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
sent_data = enron_data.sort_values('sentiment')
sent_data['level'] = '.'
for i in range(len(sent_data)):
    val= sent_data.at[i,'sentiment']
        
    if val >= 0.5 :
        sent_data.at[i,'level'] = 'e'

    if (val >= 0.1) & (val < 0.5):
        sent_data.at[i,'level'] = 'd'
         
    if (val < 0.1) & (val > -0.1) :
        sent_data.at[i,'level'] = 'c'
        
    if (val <= -0.1) & (val > -0.5):
        sent_data.at[i,'level'] = 'b' 
    
    if val <= -0.5:
        sent_data.at[i,'level'] = 'a'

In [3]:
sent_data['from email, to email'] = sent_data['fromEmail'] + ';' + sent_data['toEmail']
sent_data['from job, to job'] = sent_data['fromJobtitle'] + ';' + sent_data['toJobtitle']

jah = enron_data['toEmail'].unique()
jah2 = enron_data['toJobtitle'].unique()
jah3 = enron_data['date'].unique()
jahh = tuple(jah) + tuple(jah2)

joblist = ['Trader', 'Unkown', 'Vice President', 'Employee', 'Managing Director', 'President', 'Manager', 'Director', 'CEO', 'In House Lawyer']

sent_grouped3 = sent_data.groupby(["toJobtitle", "level"]).count()
sent_grouped3["count"] = sent_grouped3["sentiment"]
sent_matrix3 = sent_grouped3["count"].unstack()

In [4]:
sent_data

,date,fromId,fromEmail,fromJobtitle,toId,toEmail,toJobtitle,messageType,sentiment,level,"from email, to email","from job, to job"
30529,2000-11-06,113,l..gay@enron.com,Employee,96,matthew.lenhart@enron.com,Employee,TO,-1.000000,a,l..gay@enron.com;matthew.lenhart@enron.com,Employee;Employee
30527,2000-11-06,113,l..gay@enron.com,Employee,41,jay.reitmeyer@enron.com,Employee,TO,-1.000000,a,l..gay@enron.com;jay.reitmeyer@enron.com,Employee;Employee
30513,2000-11-06,113,l..gay@enron.com,Employee,146,tori.kuykendall@enron.com,Trader,TO,-1.000000,a,l..gay@enron.com;tori.kuykendall@enron.com,Employee;Trader
30378,2001-01-03,148,j.kaminski@enron.com,Manager,5,jeff.skilling@enron.com,CEO,TO,-0.736842,a,j.kaminski@enron.com;jeff.skilling@enron.com,Manager;CEO
30375,2001-01-03,148,j.kaminski@enron.com,Manager,148,j.kaminski@enron.com,Manager,CC,-0.736842,a,j.kaminski@enron.com;j.kaminski@enron.com,Manager;Manager
...,...,...,...,...,...,...,...,...,...,...,...,...
28835,2001-06-07,116,richard.shapiro@enron.com,Vice President,134,steven.j.kean@enron.com,Vice President,TO,0.520548,e,richard.shapiro@enron.com;steven.j.kean@enron.com,Vice President;Vice President
30044,2001-10-25,112,phillip.m.love@enron.com,Unknown,67,c..giron@enron.com,Employee,TO,0.547945,e,phillip.m.love@enron.com;c..giron@enron.com,Unknown;Employee
30028,2001-10-23,129,shelley.corman@enron.com,Vice President,6,lynn.blair@enron.com,Director,TO,0.917808,e,shelley.corman@enron.com;lynn.blair@enron.com,Vice President;Director
18038,2001-10-24,141,tana.jones@enron.com,Unknown,1,marie.heard@enron.com,Unknown,TO,0.931507,e,tana.jones@enron.com;marie.heard@enron.com,Unknown;Unknown


In [5]:

def f(PeopleFilter):
    
    global sent_matrix    
    sent_grouped = sent_data.groupby(["toJobtitle", "level"]).count()
    sent_grouped["count"] = sent_grouped["sentiment"]
    sent_matrix = sent_grouped["count"].unstack()
    sent_matrix = sent_matrix.fillna(0)
    
    if ";" in PeopleFilter:
        if (PeopleFilter.partition(";")[0] in joblist and PeopleFilter.partition(";")[2] in joblist):
            CertainMails = (sent_data['from job, to job'] == PeopleFilter)
            sent_data2 = sent_data[CertainMails].sort_values('sentiment')
            sent_grouped = sent_data2.groupby(["from job, to job", "level"]).count()
            sent_grouped["count"] = sent_grouped["sentiment"]
            sent_matrix = sent_grouped["count"].unstack()
        elif ("@" in PeopleFilter.partition(";")[0] and "@" in PeopleFilter.partition(";")[2]):
            CertainMails = (sent_data['from email, to email'] == PeopleFilter)
            sent_data2 = sent_data[CertainMails].sort_values('sentiment')
            sent_grouped = sent_data2.groupby(["from email, to email", "level"]).count()
            sent_grouped["count"] = sent_grouped["sentiment"]
            sent_matrix = sent_grouped["count"].unstack()
        elif (PeopleFilter.partition(";")[0] == 'to' and PeopleFilter.partition(";")[2] in joblist):
            CertainMails = (sent_data['toJobtitle'] == PeopleFilter.partition(";")[2])
            sent_data2 = sent_data[CertainMails].sort_values('sentiment')
            sent_grouped = sent_data2.groupby(["toJobtitle", "level"]).count()
            sent_grouped["count"] = sent_grouped["sentiment"]
            sent_matrix = sent_grouped["count"].unstack()
        elif (PeopleFilter.partition(";")[0] == 'from' and PeopleFilter.partition(";")[2] in joblist):
            CertainMails = (sent_data['fromJobtitle'] == PeopleFilter.partition(";")[2])
            sent_data2 = sent_data[CertainMails].sort_values('sentiment')
            sent_grouped = sent_data2.groupby(["fromJobtitle", "level"]).count()
            sent_grouped["count"] = sent_grouped["sentiment"]
            sent_matrix = sent_grouped["count"].unstack()
        elif (PeopleFilter.partition(";")[0] == 'to' and '@' in PeopleFilter.partition(";")[2]):
            CertainMails = (sent_data['toEmail'] == PeopleFilter.partition(";")[2])
            sent_data2 = sent_data[CertainMails].sort_values('sentiment')
            sent_grouped = sent_data2.groupby(["toEmail", "level"]).count()
            sent_grouped["count"] = sent_grouped["sentiment"]
            sent_matrix = sent_grouped["count"].unstack()
        elif (PeopleFilter.partition(";")[0] == 'from' and '@' in PeopleFilter.partition(";")[2]):
            CertainMails = (sent_data['fromEmail'] == PeopleFilter.partition(";")[2])
            sent_data2 = sent_data[CertainMails].sort_values('sentiment')               
            sent_grouped = sent_data2.groupby(["fromEmail", "level"]).count()
            sent_grouped["count"] = sent_grouped["sentiment"]
            sent_matrix = sent_grouped["count"].unstack()

    elif ";" not in PeopleFilter:
        if "@" in PeopleFilter:
            CertainMails = ((sent_data['fromEmail'] == PeopleFilter) | (sent_data['toEmail'] == PeopleFilter))
            sent_data2 = sent_data[CertainMails].sort_values('sentiment')
            sent_grouped = sent_data2.groupby(["from email, to email", "level"]).count()
            sent_grouped["count"] = sent_grouped["sentiment"]
            sent_matrix = sent_grouped["count"].unstack()
        elif PeopleFilter in joblist:
            CertainMails = ((sent_data['fromJobtitle'] == PeopleFilter) | (sent_data['toJobtitle'] == PeopleFilter))            
            sent_data2 = sent_data[CertainMails].sort_values('sentiment')
            sent_grouped = sent_data2.groupby(["from job, to job", "level"]).count()
            sent_grouped["count"] = sent_grouped["sentiment"]
            sent_matrix = sent_grouped["count"].unstack()
    
      
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 10))      
    ax.set_ylabel('sentiment level')
    
    return sns.heatmap(data=sent_matrix, xticklabels=['very bad', 'bad', 'neutral', 'good', 'very good'], annot = True, ax = ax)



search = interact_manual(f, PeopleFilter=widgets.Combobox(value="something;something"))

interactive(children=(Combobox(value='something;something', description='PeopleFilter'), Button(description='R…